In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import os
import time
import yfinance as yf
import sqlite3
from dotenv import load_dotenv
from datetime import datetime, timedelta



In [ ]:
load_dotenv()
api_key = os.getenv("FINNHUB_API_KEY")

In [ ]:
DATABASE_NAME = 'stocks_data.db'
conn = sqlite3.connect(DATABASE_NAME)
print(f"Connected to database: {DATABASE_NAME}")
c = conn.cursor()

DEFAULT_START_DATE = pd.to_datetime("2023-08-01")
c.execute('''
CREATE TABLE IF NOT EXISTS stock_data_combined (
    ticker TEXT, 
    date DATE, 
    open REAL, 
    high REAL, 
    low REAL, 
    close REAL, 
    volume INTEGER, 
    dividends REAL, 
    stock_splits REAL, 
    eps_estimate REAL,
    reported_eps REAL,
    surprise_percentage REAL,
    change INTEGER, 
    transactionPrice REAL, 
    number_of_buys INTEGER,
    number_of_sells INTEGER,
    number_of_gifts INTEGER
)''')
print("Table 'stock_data_combined' creation command executed.")

c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = c.fetchall()
if ('stock_data_combined',) in tables:
    print("'stock_data_combined' table exists.")
else:
    print("'stock_data_combined' table does not exist.")
conn.commit()


In [ ]:
def fetch_data_for_period(stock, from_date, to_date, api_key):
    time.sleep(1) 
    
    from_date = pd.to_datetime(from_date)
    to_date = pd.to_datetime(to_date)
    url = f'https://finnhub.io/api/v1/stock/insider-transactions?symbol={stock}&from={from_date.strftime("%Y-%m-%d")}&to={to_date.strftime("%Y-%m-%d")}&token={api_key}'
    r = requests.get(url)
    data = json.loads(r.text)

    all_data = pd.DataFrame() 

    if 'data' not in data or len(data['data']) < 2000:
        return pd.DataFrame(data=data.get('data', []))
    else:
        middle_date = from_date + (to_date - from_date) / 2
        first_half = fetch_data_for_period(stock, from_date, middle_date.strftime("%Y-%m-%d"), api_key)
        second_half = fetch_data_for_period(stock, middle_date.strftime("%Y-%m-%d"), to_date, api_key)
        all_data = pd.concat([all_data, first_half, second_half])

    return all_data

def fetch_data_from_api(stock, start_date, end_date=None, api_key=None):
    if end_date is None:
        end_date = pd.to_datetime("today")
    else:
        end_date = pd.to_datetime(end_date)

    start_date = pd.to_datetime(start_date)

    all_data = fetch_data_for_period(stock, start_date, end_date, api_key)
    return all_data


def fetch_stock_data(ticker_symbol, start_date=None, end_date=None):
    # Fetching the ticker data
    ticker_data = yf.Ticker(ticker_symbol)
    historical_data = ticker_data.history(period="max")
    historical_data.index = historical_data.index.normalize()
    earnings_dates_data = ticker_data.get_earnings_dates(limit=200)
    earnings_dates_data.index = earnings_dates_data.index.normalize()

    # If start_date and/or end_date is provided, filter the historical_data
    if start_date:
        # Ensure start_date is tz-aware
        start_date = pd.Timestamp(start_date).tz_localize(historical_data.index.tz).normalize()
        historical_data = historical_data[historical_data.index >= start_date]

    if end_date:
        # Ensure end_date is tz-aware
        end_date = pd.Timestamp(end_date).tz_localize(historical_data.index.tz).normalize()
        historical_data = historical_data[historical_data.index <= end_date]

    # Merging data
    combined_data = historical_data.merge(earnings_dates_data, left_index=True, right_index=True, how='left')
    
    return combined_data


def get_missing_date_ranges(conn, stock, start_date=None):
    # Query the database to get all dates for which we have data for the stock
    c = conn.cursor()
    c.execute('SELECT date FROM stock_data_combined WHERE ticker = ?', (stock,))
    dates_in_db = c.fetchall()
    dates_in_db = [pd.to_datetime(date[0]) for date in dates_in_db if date[0] is not None]

    # Use the provided start_date or DEFAULT_START_DATE
    if not start_date:
        start_date = DEFAULT_START_DATE
    start_date = pd.to_datetime(start_date).tz_localize(None)

    # All business days from start_date to today
    end_date = pd.to_datetime("today")
    all_dates = pd.date_range(start_date, end_date, freq='B')

    # Detect the missing dates
    missing_dates = [date for date in all_dates if date not in dates_in_db]

    # Construct date ranges
    date_ranges = []
    if missing_dates:
        start_range = missing_dates[0]
        end_range = missing_dates[0]
        
        for i in range(1, len(missing_dates)):
            if missing_dates[i] == end_range + pd.Timedelta(days=1):
                # Extend the range
                end_range = missing_dates[i]
            else:
                # Save the previous range and start a new one
                date_ranges.append((start_range, end_range))
                start_range = missing_dates[i]
                end_range = missing_dates[i]
                
        # Add the last detected range
        date_ranges.append((start_range, end_range))

    return date_ranges

In [ ]:
# Test the api
# print(fetch_data_from_api('AAPL', '2023-08-01', api_key).head())

In [ ]:
#get list of all US stock tickers
url = f'https://finnhub.io/api/v1/stock/symbol?exchange=US&token={api_key}'
r = requests.get(url)
data = json.loads(r.text)

stock_tickers = [item['symbol'] for item in data]

print(stock_tickers)

In [ ]:
stock_tickers = ["AAPL", "GILD", "TSLA"]  # Temporary shorter list of tickers to fetch data for

for stock in stock_tickers:
    missing_ranges = get_missing_date_ranges(conn, stock)
    # If no missing dates, continue to the next stock
    if not missing_ranges:
        continue
    
    for start_date, end_date in missing_ranges:
        print(f"Fetching data for {stock} from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")
        
        insider_data = fetch_data_from_api(stock, start_date, end_date, api_key)
        # Assign the stock ticker to the fetched insider data for identification
        insider_data['ticker'] = stock

        print(f"Fetched {len(insider_data)} insider transactions for {stock}...")

        stock_data = fetch_stock_data(stock, start_date, end_date) 
        stock_data.index = stock_data.index.tz_localize(None)
        
        # Filter the stock data to only include entries from the start_date to end_date
        stock_data = stock_data[start_date:end_date] 
        stock_data['ticker'] = stock
        
        print(f"Fetched stock data for {stock} from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...")

        # If insider_data isn't empty, compute the insider related columns
        if not insider_data.empty:
            insider_data['insiderPortfolioChange'] = insider_data['change'] / (insider_data['share'] - insider_data['change'])
            conditions = [
                (insider_data['change'] >= 0) & (insider_data['transactionPrice'] > 0),
                (insider_data['change'] <= 0) & (insider_data['transactionPrice'] > 0),
                (insider_data['transactionPrice'] == 0)
            ]
            values = ['Buy', 'Sale', 'Gift']
            insider_data['buyOrSale'] = np.select(conditions, values)
            insider_data['transactionDate'] = pd.to_datetime(insider_data['transactionDate'])

            # Aggregate data based on transaction date
            aggregated_data = insider_data.groupby('transactionDate').agg({
                'change': 'sum',
                'transactionPrice': 'mean',
                'buyOrSale': lambda x: x.value_counts().to_dict()
            }).reset_index()

            # Split the buyOrSale dictionary column into separate columns
            aggregated_data['Number of Buys'] = aggregated_data['buyOrSale'].apply(lambda x: x.get('Buy', 0))
            aggregated_data['Number of Sells'] = aggregated_data['buyOrSale'].apply(lambda x: x.get('Sale', 0))
            aggregated_data['Number of Gifts'] = aggregated_data['buyOrSale'].apply(lambda x: x.get('Gift', 0))
            aggregated_data.drop('buyOrSale', axis=1, inplace=True)
            # Convert 'transactionDate' column in all_data to datetime
            aggregated_data['transactionDate'] = pd.to_datetime(aggregated_data['transactionDate'])
            aggregated_data['transactionDate'] = aggregated_data['transactionDate'].dt.normalize()
        else:
            # Create an empty aggregated_data DataFrame with the necessary columns if insider_data is empty
            aggregated_data = pd.DataFrame(columns=['transactionDate', 'change', 'transactionPrice', 'Number of Buys', 'Number of Sells', 'Number of Gifts'])

        # Remove timezone from the stock_data index
        stock_data.index = stock_data.index.tz_localize(None)
        stock_data = stock_data.reset_index().rename(columns={'index': 'date'})

        # Merge on the 'date' column
        combined_data = pd.merge(stock_data, aggregated_data, left_on='date', right_on='transactionDate', how='left')
        combined_data['date'] = combined_data['date'].dt.normalize()
        combined_data.drop('transactionDate', axis=1, inplace=True)  # Drop the redundant transactionDate column

        # Renaming columns
        renamed_columns = {
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Volume": "volume",
            "Dividends": "dividends",
            "Stock Splits": "stock_splits",
            "EPS Estimate": "eps_estimate",
            "Reported EPS": "reported_eps",
            "Surprise(%)": "surprise_percentage",
            "Number of Buys": "number_of_buys",
            "Number of Sells": "number_of_sells",
            "Number of Gifts": "number_of_gifts"
        }

        # Write the combined data to the SQLite database
        combined_data = combined_data.rename(columns=renamed_columns)
        combined_data.to_sql('stock_data_combined', conn, if_exists='append', index=False)
        # print(combined_data.head())
    print(f"Written combined data for {stock} to database.\n")

In [ ]:
# Test deleting database records to see if the previous cell will find them and handle them
c.execute(f"DELETE FROM stock_data_combined WHERE date = '2023-08-30 00:00:00'")
c.execute(f"DELETE FROM stock_data_combined WHERE date = '2023-08-24 00:00:00'")
c.execute(f"DELETE FROM stock_data_combined WHERE date = '2023-08-25 00:00:00'")
c.execute(f"DELETE FROM stock_data_combined WHERE date = '2023-08-01 00:00:00'")
conn.commit()

In [ ]:
# Query a few rows from the database to inspect the data
c.execute('SELECT * FROM stock_data_combined LIMIT 2000')
rows = c.fetchall()

# Get column names from cursor description
column_names = [description[0] for description in c.description]
print(column_names)
for row in rows:
    print(row)


In [164]:
# Close database connection
conn.close()